In [1]:
from coding_library import *

In [2]:
from scipy import stats

stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = stats.linregress(stock_returns, mkt_returns)

beta, alpha

(0.5077431878770808, -0.008481900352462384)

In [3]:
import numpy as np
import statsmodels.api as sm
# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8) for i in range(num_periods)])
# Filter the data
y_values = all_values[:, 0] # First column values as Y
x_values = all_values[:, 1:] # All other values as X
x_values = sm.add_constant(x_values) # Include the intercept
results = sm.OLS(y_values, x_values).fit() # Regress and fit the model

In [4]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     3.024
Date:                Sun, 10 Mar 2024   Prob (F-statistic):              0.417
Time:                        14:40:49   Log-Likelihood:                 11.375
No. Observations:                   9   AIC:                            -6.751
Df Residuals:                       1   BIC:                            -5.173
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0572      0.335     -0.171      0.8

/Users/bernardocohen/repos/python/mastering_python_for_finance/venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1971: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  k, _ = kurtosistest(a, axis)


In [5]:
results.params

array([-0.05718742, -1.89474135,  1.64626241, -1.29004789,  2.15425536,
        0.07312914,  0.38562831, -0.00312286])

In [6]:
import pulp
x = pulp.LpVariable("x", lowBound=0)
y = pulp.LpVariable("y", lowBound=0)
problem = pulp.LpProblem("A simple maximization objective",
pulp.LpMaximize)
problem += 3*x + 2*y, "The objective function"
problem += 2*x + y <= 100, "1st constraint"
problem += x + y <= 80, "2nd constraint"
problem += x <= 40, "3rd constraint"
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bernardocohen/repos/python/mastering_python_for_finance/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/09/zcndpz6n6jxbcrkqzlh7bclm0000gn/T/53f3be5d15b44e2dbdc534197090eacc-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/09/zcndpz6n6jxbcrkqzlh7bclm0000gn/T/53f3be5d15b44e2dbdc534197090eacc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 16 RHS
At line 20 BOUNDS
At line 21 ENDATA
Problem MODEL has 3 rows, 2 columns and 5 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-1) rows, 2 (0) columns and 4 (-1) elements
0  Obj -0 Dual inf 4.999998 (2)
2  Obj 180
Optimal - objective value 180
After Postsolve, objective 180, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 180 - 2 iterations time 0.002, Presolve 0.00
Opt

/Users/bernardocohen/repos/python/mastering_python_for_finance/venv/lib/python3.11/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [7]:
for variable in problem.variables():
    print(f"{variable.name} = {variable.value()}")

x = 20.0
y = 60.0


In [8]:
# import pulp
dealers = ["X", "Y", "Z"]
variable_costs = {"X": 500,
"Y": 350,
"Z": 450}
fixed_costs = {"X": 4000,
"Y": 2000,
"Z": 6000}
# Define PuLP variables to solve
quantities = pulp.LpVariable.dicts("quantity",
dealers,
lowBound=0,
cat=pulp.LpInteger)
is_orders = pulp.LpVariable.dicts("orders",
dealers,
cat=pulp.LpBinary)

"""
This is an example of implementing an integer programming model with
binary
variables
the wrong way.
"""
#### Initialize the model with constraints
model = pulp.LpProblem("A cost minimization problem",
pulp.LpMinimize)
model += sum([(variable_costs[i] * quantities[i] +
fixed_costs[i])*is_orders[i] for i in dealers]), \
"Minimize portfolio cost"
model += sum([quantities[i] for i in dealers]) == 150, "Total contracts required"

model += 30 <= quantities["X"] <= 100, "Boundary of total volume of X"
model += 30 <= quantities["Y"] <= 90, "Boundary of total volume of Y"
model += 30 <= quantities["Z"] <= 70, "Boundary of total volume of Z"
model.solve()

In [9]:
"""
This is an example of implementing an IP model
with binary variables the correct way.
"""
# Initialize the model with constraints
model = pulp.LpProblem("A cost minimization problem",
pulp.LpMinimize)
model += sum([variable_costs[i]*quantities[i] +
fixed_costs[i]*is_orders[i] for i in dealers]), \
"Minimize portfolio cost"
model += sum([quantities[i] for i in dealers]) == 150, \
"Total contracts required"
model += is_orders["X"]*30 <= quantities["X"] <= \
is_orders["X"]*100, "Boundary of total volume of X"
model += is_orders["Y"]*30 <= quantities["Y"] <= \
is_orders["Y"]*90, "Boundary of total volume of Y"
model += is_orders["Z"]*30 <= quantities["Z"] <= \
is_orders["Z"]*70, "Boundary of total volume of Z"
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/bernardocohen/repos/python/mastering_python_for_finance/venv/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/09/zcndpz6n6jxbcrkqzlh7bclm0000gn/T/76d23e6ac2e54f57a8565c206ceac760-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/09/zcndpz6n6jxbcrkqzlh7bclm0000gn/T/76d23e6ac2e54f57a8565c206ceac760-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 37 RHS
At line 42 BOUNDS
At line 49 ENDATA
Problem MODEL has 4 rows, 6 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 65642.9 - 0.00 seconds
Cgl0004I processed model has 4 rows, 6 columns (6 integer (3 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 49.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.142857
Cbc0038I Solutio

1

In [10]:
print("Minimization results")
for variable in model.variables():
    print(f"{variable.name} = {variable.value()}")

print(f"Total cost: {pulp.value(model.objective)}")

Minimization results
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


In [11]:
""" Linear algebra with NumPy matrices """
import numpy as np
A = np.array([[2, 1, 1],
[1, 3, 2],
[1, 0, 0]])
B = np.array([4, 5, 6])

In [12]:
print(np.linalg.solve(A, B))

[  6.  15. -23.]


In [13]:
""" LU decomposition with SciPy """
import scipy.linalg as linalg
import numpy as np
A = np.array([[2., 1., 1.],
[1., 3., 2.],
[1., 0., 0.]])
B = np.array([4., 5., 6.])
LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)
x

array([  6.,  15., -23.])

In [14]:
P, L, U = linalg.lu(A)
P, L, U

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[ 1. ,  0. ,  0. ],
        [ 0.5,  1. ,  0. ],
        [ 0.5, -0.2,  1. ]]),
 array([[ 2. ,  1. ,  1. ],
        [ 0. ,  2.5,  1.5],
        [ 0. ,  0. , -0.2]]))

In [15]:
""" Cholesky decomposition with NumPy """
import numpy as np
A = np.array([[10., -1., 2., 0.],
[-1., 11., -1., 3.],
[2., -1., 10., -1.],
[0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
L = np.linalg.cholesky(A)
L

array([[ 3.16227766,  0.        ,  0.        ,  0.        ],
       [-0.31622777,  3.3015148 ,  0.        ,  0.        ],
       [ 0.63245553, -0.24231301,  3.08889696,  0.        ],
       [ 0.        ,  0.9086738 , -0.25245792,  2.6665665 ]])

In [16]:
print(np.dot(L, L.T.conj())) # A=L.L*

[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]


In [17]:
y = np.linalg.solve(L, B) # L.L*.x=B; When L*.x=y, then L.y=B
x = np.linalg.solve(L.T.conj(), y) # x=L*'.y
x

array([ 1.,  2., -1.,  1.])

In [18]:
print(np.mat(A) * np.mat(x).T) # B=Ax

[[  6.]
 [ 25.]
 [-11.]
 [ 15.]]


In [19]:
""" QR decomposition with scipy """
import scipy.linalg as linalg
import numpy as np
A = np.array([
[2., 1., 1.],
[1., 3., 2.],
[1., 0., 0]])
B = np.array([4., 5., 6.])
Q, R = linalg.qr(A) # QR decomposition
y = np.dot(Q.T, B) # Let y=Q'.B
x = linalg.solve(R, y) # Solve Rx=y
x

array([  6.,  15., -23.])

In [20]:
A = np.array([[10., -1., 2., 0.],
[-1., 11., -1., 3.],
[2., -1., 10., -1.],
[0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25

x = jacobi(A,B,n)
x

array([ 1.,  2., -1.,  1.])

In [21]:
A = np.array([[10., -1., 2., 0.],
[-1., 11., -1., 3.],
[2., -1., 10., -1.],
[0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25

x = gauss(A,B,n)
x

array([ 1.,  2., -1.,  1.])